In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageOps
import PIL
import torch, torchvision
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from dataset import *
import random
import math
import seaborn
from  matplotlib import pyplot
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import time

In [2]:
input_size = 10       # The image size = 28 x 28 = 784
hidden_size = 500      # The number of nodes at the hidden layer
num_classes = 5       # The number of output classes. In this case, from 0 to 9
num_epochs = 10         # The number of times entire dataset is trained
batch_size = 256       # The size of input data took for one iteration
learning_rate = 0.0001  # The speed of convergence

In [3]:
data_transform = transforms.Compose([
        transforms.RandomResizedCrop(200, scale=(1, 1), ratio=(1, 1)),
        transforms.RandomRotation((-90,90)),
        torchvision.transforms.RandomVerticalFlip(p=0.5),
        torchvision.transforms.RandomHorizontalFlip(p=0.5),
#         torchvision.transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0, hue=0),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.3019],
                             std=[0.1909])
    ])

In [4]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 10 (input data) -> 500 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, hidden_size) # 2nd Full-Connected Layer: 500 (hidden node) -> 5 (output class)
#         self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
#         self.fc3 = nn.Linear(hidden_size, num_classes) # 3rd Full-Connected Layer: 500 (hidden node) -> 5 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
#         out = self.relu(out)
#         out = self.fc3(out)
        return out

In [5]:
net = Net(input_size, hidden_size, num_classes)

In [6]:
trainset = defectDataset_df(df = split_and_sample(method = 'hard',n_samples = 1995, non_pos_ratio=1), window_size = window_size,
                                             transforms=data_transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                             batch_size=batch_size, shuffle=True,
                                             num_workers=8, drop_last=True)
print("trainloader ready!")

testset = defectDataset_df(df = split_and_sample(df_train = pd.read_csv('/home/rliu/yolo2/v2_pytorch_yolo2/data/an_data/VOCdevkit/VOC2007/csv_labels/test.csv', sep=" "),
                                                      method = 'hard',n_samples = 800), window_size = window_size, transforms=data_transform)
testloader = torch.utils.data.DataLoader(testset,
                                             batch_size=batch_size, shuffle=True,
                                             num_workers=8)
print("testloader ready!")


trainloader ready!
testloader ready!


In [7]:
use_gpu = torch.cuda.is_available()

if use_gpu:
    print("GPU in use")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

classes = ["pos","neg","pos_o","nuc","non"]
num_of_classes = len(classes)

model_uniform = torch.load('/home/rliu/defect_classifier/models/python/res34_600epo_uniform_01-07-18.model')
model_uniform.eval()
model_hard = torch.load('/home/rliu/defect_classifier/models/python/res34_600epo_hard_01-07-18.model')
model_hard.eval()


if use_gpu:
#     model_uniform = torch.nn.DataParallel(model_uniform)
    model_uniform.to(device)
#     model_hard = torch.nn.DataParallel(model_hard)
    model_hard.to(device)
    net.to(device)

GPU in use


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [9]:
since = time.time()
best_model_wts = net.state_dict()
best_acc = 0.0
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    model_uniform.train(True)
    model_hard.train(True)
    net.train(True)
    running_loss = 0.0
    running_corrects = 0
    for data in trainloader:
        
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        with torch.no_grad():
            outputs_uniform = model_uniform(inputs)
            outputs_hard = model_hard(inputs)
        outputs_in = torch.cat((outputs_uniform, outputs_hard), dim=1)
        outputs_out = net(outputs_in)
        _, preds = torch.max(outputs_out.data, 1)
        loss = criterion(outputs_out, labels)

        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes

#         if (i+1) % 100 == 0:                              # Logging
#             print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
#                  %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item()))

        
        # statistics
        iter_loss = loss.item()
        correct = torch.sum(preds == labels.data).item()
        batch_accuracy = correct / batch_size
        running_loss += loss.item()
        running_corrects_tensor = torch.sum(preds == labels.data)
        running_corrects += running_corrects_tensor.item()        
        epoch_loss = running_loss / len(trainset)
        epoch_acc = running_corrects / len(trainset)

        print('{} Loss: {:.4f} Acc: {:.4f} batch_loss: {:.4f} correct: {:d} batch_accuracy: {:.4f}'.format(
            "train", epoch_loss, epoch_acc, iter_loss, correct, batch_accuracy))
        
    correct = 0
    total = 0
    class_correct = list(0. for i in range(5))
    class_total = list(0. for i in range(5))    
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs_uniform = model_uniform(inputs)
            outputs_hard = model_hard(inputs)
            outputs_in = torch.cat((outputs_uniform, outputs_hard), dim=1)
            outputs_out = net(outputs_in)
            _, predicted = torch.max(outputs_out.data, 1)
            c = (predicted == labels).squeeze()
            for i in range(batch_size):
                if len(labels) == batch_size:
                    label = labels[i]
                    class_correct[label] += c[i].item()
                    class_total[label] += 1
                    correct += c[i].item()
                    total += 1
#             if len(labels) == batch_size:
#                 total += labels.size(0)
#                 correct += (predicted == labels).sum().item()
    #         print(predicted)
    #         print(labels)
    #       print('processed: %d' % total)
    #       print('correct: %d' % correct)
        print('Accuracy of the network on the test images: %.5f %%' % (100 * correct / total))
        for i in range(5):
            print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))
        print('class total: ',class_total)
        print('class correct: ',class_correct)
        print('total: ', total)
        print('correct: ', correct)
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
print('Best val Acc: {:4f}'.format(best_acc))

# load best model weights
net.load_state_dict(best_model_wts)

Epoch 0/9
----------
train Loss: 0.0011 Acc: 0.0000 batch_loss: 11.1480 correct: 0 batch_accuracy: 0.0000
train Loss: 0.0022 Acc: 0.0000 batch_loss: 10.9080 correct: 0 batch_accuracy: 0.0000
train Loss: 0.0033 Acc: 0.0000 batch_loss: 10.7185 correct: 0 batch_accuracy: 0.0000
train Loss: 0.0043 Acc: 0.0000 batch_loss: 10.3903 correct: 0 batch_accuracy: 0.0000
train Loss: 0.0053 Acc: 0.0000 batch_loss: 10.0797 correct: 0 batch_accuracy: 0.0000
train Loss: 0.0063 Acc: 0.0000 batch_loss: 9.9035 correct: 0 batch_accuracy: 0.0000
train Loss: 0.0073 Acc: 0.0000 batch_loss: 9.6065 correct: 0 batch_accuracy: 0.0000
train Loss: 0.0082 Acc: 0.0000 batch_loss: 9.3802 correct: 0 batch_accuracy: 0.0000
train Loss: 0.0091 Acc: 0.0000 batch_loss: 9.1168 correct: 0 batch_accuracy: 0.0000
train Loss: 0.0100 Acc: 0.0000 batch_loss: 8.8565 correct: 0 batch_accuracy: 0.0000
train Loss: 0.0109 Acc: 0.0000 batch_loss: 8.5703 correct: 0 batch_accuracy: 0.0000
train Loss: 0.0117 Acc: 0.0000 batch_loss: 8.1454 

train Loss: 0.0005 Acc: 0.3413 batch_loss: 0.2804 correct: 249 batch_accuracy: 0.9727
train Loss: 0.0006 Acc: 0.3655 batch_loss: 0.3246 correct: 242 batch_accuracy: 0.9453
train Loss: 0.0006 Acc: 0.3896 batch_loss: 0.3249 correct: 240 batch_accuracy: 0.9375
train Loss: 0.0006 Acc: 0.4148 batch_loss: 0.2760 correct: 252 batch_accuracy: 0.9844
train Loss: 0.0007 Acc: 0.4395 batch_loss: 0.3109 correct: 246 batch_accuracy: 0.9609
train Loss: 0.0007 Acc: 0.4636 batch_loss: 0.2825 correct: 240 batch_accuracy: 0.9375
train Loss: 0.0007 Acc: 0.4885 batch_loss: 0.2336 correct: 249 batch_accuracy: 0.9727
train Loss: 0.0007 Acc: 0.5130 batch_loss: 0.2850 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0008 Acc: 0.5372 batch_loss: 0.3217 correct: 242 batch_accuracy: 0.9453
train Loss: 0.0008 Acc: 0.5613 batch_loss: 0.3068 correct: 240 batch_accuracy: 0.9375
train Loss: 0.0008 Acc: 0.5859 batch_loss: 0.2675 correct: 245 batch_accuracy: 0.9570
train Loss: 0.0009 Acc: 0.6104 batch_loss: 0.2886 corr

train Loss: 0.0005 Acc: 0.6404 batch_loss: 0.2211 correct: 243 batch_accuracy: 0.9492
train Loss: 0.0005 Acc: 0.6651 batch_loss: 0.1632 correct: 246 batch_accuracy: 0.9609
train Loss: 0.0005 Acc: 0.6898 batch_loss: 0.1576 correct: 247 batch_accuracy: 0.9648
train Loss: 0.0005 Acc: 0.7145 batch_loss: 0.1909 correct: 246 batch_accuracy: 0.9609
train Loss: 0.0005 Acc: 0.7387 batch_loss: 0.1964 correct: 242 batch_accuracy: 0.9453
train Loss: 0.0005 Acc: 0.7634 batch_loss: 0.1603 correct: 246 batch_accuracy: 0.9609
train Loss: 0.0006 Acc: 0.7880 batch_loss: 0.2109 correct: 245 batch_accuracy: 0.9570
train Loss: 0.0006 Acc: 0.8126 batch_loss: 0.1510 correct: 246 batch_accuracy: 0.9609
train Loss: 0.0006 Acc: 0.8370 batch_loss: 0.1874 correct: 243 batch_accuracy: 0.9492
train Loss: 0.0006 Acc: 0.8616 batch_loss: 0.1769 correct: 245 batch_accuracy: 0.9570
train Loss: 0.0006 Acc: 0.8865 batch_loss: 0.1444 correct: 249 batch_accuracy: 0.9727
train Loss: 0.0006 Acc: 0.9112 batch_loss: 0.1309 corr

train Loss: 0.0006 Acc: 0.9365 batch_loss: 0.0968 correct: 250 batch_accuracy: 0.9766
Accuracy of the network on the test images: 82.31771 %
Accuracy of   pos : 86 %
Accuracy of   neg : 89 %
Accuracy of pos_o : 74 %
Accuracy of   nuc : 73 %
Accuracy of   non : 88 %
class total:  [765.0, 774.0, 771.0, 771.0, 759.0]
class correct:  [658.0, 690.0, 571.0, 568.0, 674.0]
total:  3840
correct:  3161
Epoch 7/9
----------
train Loss: 0.0000 Acc: 0.0249 batch_loss: 0.1486 correct: 248 batch_accuracy: 0.9688
train Loss: 0.0000 Acc: 0.0497 batch_loss: 0.0963 correct: 248 batch_accuracy: 0.9688
train Loss: 0.0000 Acc: 0.0741 batch_loss: 0.1556 correct: 243 batch_accuracy: 0.9492
train Loss: 0.0001 Acc: 0.0985 batch_loss: 0.1747 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0001 Acc: 0.1230 batch_loss: 0.1388 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0001 Acc: 0.1479 batch_loss: 0.1275 correct: 248 batch_accuracy: 0.9688
train Loss: 0.0001 Acc: 0.1726 batch_loss: 0.1323 correct: 247 batc

train Loss: 0.0001 Acc: 0.1970 batch_loss: 0.1683 correct: 242 batch_accuracy: 0.9453
train Loss: 0.0001 Acc: 0.2215 batch_loss: 0.1495 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0001 Acc: 0.2465 batch_loss: 0.0911 correct: 250 batch_accuracy: 0.9766
train Loss: 0.0001 Acc: 0.2713 batch_loss: 0.1325 correct: 247 batch_accuracy: 0.9648
train Loss: 0.0002 Acc: 0.2959 batch_loss: 0.1301 correct: 246 batch_accuracy: 0.9609
train Loss: 0.0002 Acc: 0.3206 batch_loss: 0.1508 correct: 246 batch_accuracy: 0.9609
train Loss: 0.0002 Acc: 0.3449 batch_loss: 0.1808 correct: 242 batch_accuracy: 0.9453
train Loss: 0.0002 Acc: 0.3696 batch_loss: 0.1205 correct: 247 batch_accuracy: 0.9648
train Loss: 0.0002 Acc: 0.3941 batch_loss: 0.1574 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0002 Acc: 0.4185 batch_loss: 0.1530 correct: 244 batch_accuracy: 0.9531
train Loss: 0.0003 Acc: 0.4431 batch_loss: 0.1700 correct: 245 batch_accuracy: 0.9570
train Loss: 0.0003 Acc: 0.4683 batch_loss: 0.0755 corr

In [ ]:
preds

In [81]:
print(inputs.shape)
print(outputs_uniform.shape)
print(outputs_in.shape)
print(outputs_out.shape)

torch.Size([256, 1, 200, 200])
torch.Size([256, 5])
torch.Size([256, 10])
torch.Size([256, 5])


In [62]:
criterion = nn.CrossEntropyLoss()

In [63]:
use_gpu = torch.cuda.is_available()

if use_gpu:
    print("GPU in use")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

classes = ["pos","neg","pos_o","nuc","non"]
num_of_classes = len(classes)

model_uniform = torch.load('/home/rliu/defect_classifier/models/python/res34_600epo_uniform_01-07-18.model')
model_uniform.eval()
model_hard = torch.load('/home/rliu/defect_classifier/models/python/res34_600epo_hard_01-07-18.model')
model_hard.eval()
if use_gpu:
#     model_uniform = torch.nn.DataParallel(model_uniform)
    model_uniform.to(device)
#     model_hard = torch.nn.DataParallel(model_hard)
    model_hard.to(device)
    
    
batch_size = 5
testset = defectDataset_df(df = split_and_sample(df_train = pd.read_csv('/home/rliu/yolo2/v2_pytorch_yolo2/data/an_data/VOCdevkit/VOC2007/csv_labels/train.csv', sep=" "),
                                                      method = 'hard',n_samples = 500), window_size = window_size, transforms=data_transform)
testloader = torch.utils.data.DataLoader(testset,
                                             batch_size=batch_size, shuffle=True,
                                             num_workers=8)
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
images, labels = images.to(device), labels.to(device)
outputs_uniform = model_uniform(images)
outputs_hard = model_hard(images)
outputs = torch.cat((outputs_uniform, outputs_hard), dim=1)
_, preds = torch.max(outputs.data, 1)
loss = criterion(outputs, labels)

GPU in use


In [64]:
print(outputs_uniform)
print(outputs_hard)
print(loss)

tensor([[-12.3108,   3.8134, -12.1610,  -8.9865, -18.6308],
        [  4.1426, -12.6464, -16.8836, -13.2110, -13.5861],
        [ -9.9046, -12.1545,  -9.0868,   2.3950, -12.2668],
        [ -5.9917,   3.7852,  -9.3418, -10.6862, -11.3693],
        [  4.1033,  -7.9563,  -6.8923, -13.0635,  -7.5087]], device='cuda:0',
       grad_fn=<GatherBackward>)
tensor([[-10.9020,   2.6503, -13.7712, -12.7975, -13.3193],
        [  3.1850, -11.8032, -13.6199, -11.8008, -12.5677],
        [-13.4012, -12.1915, -14.3676,   1.9575, -15.2399],
        [ -7.2337,   2.6256, -18.3268, -10.4256,  -9.0024],
        [  3.2153, -10.9220, -13.6908, -12.3731, -11.7074]], device='cuda:0',
       grad_fn=<GatherBackward>)
tensor(0.3425, device='cuda:0', grad_fn=<NllLossBackward>)


In [54]:
labels

tensor([3, 0, 0, 3, 1])

In [3]:
train_dataset = datasets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

test_dataset = datasets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

In [51]:
torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

device(type='cuda', index=0)

In [4]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [5]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 784 (input data) -> 500 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes) # 2nd Full-Connected Layer: 500 (hidden node) -> 10 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [6]:
net = Net(input_size, hidden_size, num_classes)

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

Net(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [16]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):   # Load a batch of images with its (index, data, class)
        images = Variable(images.view(-1, 28*28))         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        labels = Variable(labels)
        
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = net(images)                             # Forward pass: compute the output class given a image
        loss = criterion(outputs, labels)                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        
        if (i+1) % 100 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.2542
Epoch [1/5], Step [200/600], Loss: 0.1384
Epoch [1/5], Step [300/600], Loss: 0.1731
Epoch [1/5], Step [400/600], Loss: 0.1142
Epoch [1/5], Step [500/600], Loss: 0.0950
Epoch [1/5], Step [600/600], Loss: 0.1930
Epoch [2/5], Step [100/600], Loss: 0.1513
Epoch [2/5], Step [200/600], Loss: 0.1833
Epoch [2/5], Step [300/600], Loss: 0.1233
Epoch [2/5], Step [400/600], Loss: 0.1183
Epoch [2/5], Step [500/600], Loss: 0.0871
Epoch [2/5], Step [600/600], Loss: 0.0677
Epoch [3/5], Step [100/600], Loss: 0.0196
Epoch [3/5], Step [200/600], Loss: 0.0825
Epoch [3/5], Step [300/600], Loss: 0.0880
Epoch [3/5], Step [400/600], Loss: 0.0757
Epoch [3/5], Step [500/600], Loss: 0.0220
Epoch [3/5], Step [600/600], Loss: 0.0282
Epoch [4/5], Step [100/600], Loss: 0.0166
Epoch [4/5], Step [200/600], Loss: 0.0539
Epoch [4/5], Step [300/600], Loss: 0.0231
Epoch [4/5], Step [400/600], Loss: 0.0305
Epoch [4/5], Step [500/600], Loss: 0.0996
Epoch [4/5], Step [600/600], Loss:

In [15]:
loss.item()

0.22127796709537506

In [ ]:
    use_gpu = torch.cuda.is_available()

    if use_gpu:
        print("GPU in use")

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    classes = ["pos","neg","pos_o","nuc","non"]
    num_of_classes = len(classes)

    model_uniform = torch.load('/home/rliu/defect_classifier/models/python/res34_600epo_uniform_01-07-18.model')
    model_uniform.eval()
    model_hard = torch.load('/home/rliu/defect_classifier/models/python/res34_600epo_uniform_01-07-18.model')
    model_hard.eval()
    if use_gpu:
        model_uniform = torch.nn.DataParallel(uniform)
        model_uniform.to(device)
        model_uniform = torch.nn.DataParallel(uniform)
        model_uniform.to(device)

In [18]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    images, labels = images.to(device), labels.to(device)
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
    total += labels.size(0)                    # Increment the total count
    correct += (predicted == labels).sum()     # Increment the correct count
    
print('Accuracy of the network on the 10K test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10K test images: 97 %
